In [1]:
import torch
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "gpu")
device

device(type='cuda', index=0)

In [32]:
model_path= 'MNV3_small_weights_cleaned_100.pth'

In [33]:
model =torchvision.models.mobilenet_v3_small(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

# Define constants
NUM_CLASSES = 8
INPUT_FEATURES = 576
HIDDEN_FEATURES = 1024
OUTPUT_FEATURES = 1280

# Define the custom classifier
classifier = nn.Sequential(
    nn.Linear(INPUT_FEATURES, HIDDEN_FEATURES, bias=True),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(HIDDEN_FEATURES, OUTPUT_FEATURES, bias=True),
    nn.Linear(OUTPUT_FEATURES, NUM_CLASSES, bias=True),  # Output layer with 8 classes
    nn.Softmax(dim=1)
)

# Replace the default classifier with the custom one
model.classifier = classifier
model.load_state_dict(torch.load(model_path))
model.eval().to(device)

# Sample input (adjust according to your model's input shape)
dummy_input = torch.randn(1, 3, 224, 224).cuda()

torch.onnx.export(model, dummy_input, "mobilenetV3_gpu_pth_v8.onnx", opset_version=8)


In [35]:
model_path='MNV3_small_weights_cleaned_100.pth'

model =torchvision.models.mobilenet_v3_small(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# Define constants
NUM_CLASSES = 8
INPUT_FEATURES = 576
HIDDEN_FEATURES = 1024
OUTPUT_FEATURES = 1280

# Define the custom classifier
classifier = nn.Sequential(
    nn.Linear(INPUT_FEATURES, HIDDEN_FEATURES, bias=True),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(HIDDEN_FEATURES, OUTPUT_FEATURES, bias=True),
    nn.Linear(OUTPUT_FEATURES, NUM_CLASSES, bias=True),  # Output layer with 8 classes
    nn.Softmax(dim=1)
)

# Replace the default classifier with the custom one
model.classifier = classifier
model.load_state_dict(torch.load(model_path))
model.eval()
eyeGazeClasses = ['Eyes Closed', 'Forward', 'Left Mirror', 'Radio', 
                  'Rearview', 'Right Mirror', 'Shoulder', 'Speedometer']

In [38]:
import os
import torchvision.transforms as transforms
import cv2
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def preprocess_image(image_path):
    pil_image = Image.fromarray(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    width, height = pil_image.size
    cropped_image = pil_image.crop((0, 0, width, 125))
    input_image = transform(cropped_image).unsqueeze(0)  # Add batch dimension
    return input_image

#the test images
test_dir = "Test Data"

# counters for accuracy calculation
total_images = 0
correct_predictions = 0

misclassified_images = []

for class_name in os.listdir(test_dir):
    class_dir = os.path.join(test_dir, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            input_image = preprocess_image(image_path)

            #make prediction
            output = model(input_image)
            _, predicted = torch.max(output.data, 1)
            predicted_class = eyeGazeClasses[predicted.item()]

            #if the prediction is correct
            total_images += 1
            if predicted_class == class_name:
                correct_predictions += 1
            else:
                misclassified_images.append((image_name, class_name, predicted_class))

#calculate accuracy
accuracy = correct_predictions / total_images * 100

#results
print(f"Accuracy: {accuracy:.2f}%")
print("Misclassified Images:")
for image_name, true_class, predicted_class in misclassified_images:
    print(f"Image: {image_name}, True Class: {true_class}, Predicted Class: {predicted_class}")

Accuracy: 70.13%
Misclassified Images:
Image: cropped2_180.png, True Class: Eyes Closed, Predicted Class: Forward
Image: cropped2_185.png, True Class: Eyes Closed, Predicted Class: Forward
Image: cropped2_265.png, True Class: Eyes Closed, Predicted Class: Rearview
Image: cropped2_270.png, True Class: Eyes Closed, Predicted Class: Forward
Image: cropped2_275.png, True Class: Eyes Closed, Predicted Class: Forward
Image: cropped2_280.png, True Class: Eyes Closed, Predicted Class: Forward
Image: cropped2_285.png, True Class: Eyes Closed, Predicted Class: Speedometer
Image: cropped2_290.png, True Class: Eyes Closed, Predicted Class: Forward
Image: cropped2_350.png, True Class: Eyes Closed, Predicted Class: Right Mirror
Image: cropped2_410.png, True Class: Eyes Closed, Predicted Class: Forward
Image: cropped2_435.png, True Class: Eyes Closed, Predicted Class: Right Mirror
Image: cropped2_440.png, True Class: Eyes Closed, Predicted Class: Right Mirror
Image: cropped2_445.png, True Class: Eyes